## In this code we are going to load pretrained image classification networks 
- ResNet50
- VGG16
- VGG19

## Then using a **pretrained network**, **feature extraction** and visualization is conducted via **t-SNE**

![deepnets](https://imgur.com/nyYh5xH.jpg)


In [ ]:
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions
from keras import backend as K
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

In [ ]:
!wget -nv https://images.immediate.co.uk/production/volatile/sites/4/2018/08/iStock_000044061370_Medium-fa5f8aa.jpg -O cat.jpg

img = Image.open('./cat.jpg')
img

## Classify ImageNet classes with ResNet50

In [ ]:
model = ResNet50(weights='imagenet')

img_path = './cat.jpg'
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
print('image.img_to_array: ', x.shape, np.max(x), np.min(x))
x = np.expand_dims(x, axis=0)
print('expand_dims: ', x.shape, np.max(x), np.min(x))
x = preprocess_input(x)
print('preprocess_input: ', x.shape, np.max(x), np.min(x))
print(model.summary())

In [ ]:
preds = model.predict(x)
print('Predicted:', decode_predictions(preds, top=3)[0])

## Extract features with VGG16

In [ ]:
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
import numpy as np

model = VGG16(weights='imagenet', include_top=False)
model.summary()

In [ ]:
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

features = model.predict(x)
print(features.shape)

## Use VGG to extract features for arbitrary CIFAR-10 images 

![CIFAR-10](https://i2.wp.com/appliedmachinelearning.blog/wp-content/uploads/2018/03/cifar2.jpg?resize=427%2C325&ssl=1)

In [ ]:
from sklearn.manifold import TSNE

In [ ]:
from keras.datasets import cifar10

# The data, split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

y_test_indx = np.squeeze(y_test)

x_test = x_test[y_test_indx < 3]   
y_test = y_test[y_test_indx < 3] 

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

print('y_test.shape:', y_test.shape)

In [ ]:
# plot samples 
for i in range(9):
	plt.subplot(330 + 1 + i)
	plt.imshow(x_test[i])
plt.show()

In [ ]:
x = preprocess_input(x_test)

features = model.predict(x)
print(features.shape)
feats = np.squeeze(features)

In [ ]:
# t-SNE
tsne = TSNE(n_components=2, perplexity=30, verbose=2).fit_transform(feats)

In [ ]:
y_test_color = np.squeeze(y_test)

plt.figure(figsize=(15, 15))
plt.scatter(tsne[:, 0], tsne[:, 1], c=y_test_color)

plt.colorbar()
plt.show()

In [ ]:
# http://alexanderfabisch.github.io/t-sne-in-scikit-learn.html
def plot_dataset(X, y, X_embedded, name, min_dist=10.0):
    fig = plt.figure(figsize=(10, 10))
    ax = plt.axes(frameon=False)
    plt.subplots_adjust(left=0.0, bottom=0.0, right=1.0, top=0.9,
                    wspace=0.0, hspace=0.0)
    
    plt.scatter(X_embedded[:, 0], X_embedded[:, 1], c=y)
    if min_dist is not None:
        from matplotlib import offsetbox
        shown_images = np.array([[15., 15.]])
        indices = np.arange(X_embedded.shape[0])
        np.random.shuffle(indices)
        for i in indices[:5000]:
            dist = np.sum((X_embedded[i] - shown_images) ** 2, 1)
            if np.min(dist) < min_dist:
                continue
            shown_images = np.r_[shown_images, [X_embedded[i]]]
            imagebox = offsetbox.AnnotationBbox(
                offsetbox.OffsetImage(X[i].reshape(32, 32, 3)), X_embedded[i])
            ax.add_artist(imagebox)

In [ ]:
plot_dataset(x_test[:150], y_test_color[:150], tsne[:150], "t-SNE", min_dist=20.0)

## Extract features from an arbitrary intermediate layer with VGG19

In [ ]:
from keras.applications.vgg19 import VGG19
from keras.preprocessing import image
from keras.applications.vgg19 import preprocess_input
from keras.models import Model
import numpy as np

base_model = VGG19(weights='imagenet')

In [ ]:
model = Model(inputs=base_model.input, outputs=base_model.get_layer('block4_pool').output)

In [ ]:
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

block4_pool_features = model.predict(x)
print(block4_pool_features.shape)